In [0]:
from pyspark import SparkContext, SparkConf

In [0]:
abc_news = 'dbfs:/FileStore/shared_uploads/michaelkennethkingston@gmail.com/abcnews.txt'
stop_words = 'dbfs:/FileStore/shared_uploads/michaelkennethkingston@gmail.com/stopwords.txt'
output = 'dbfs:/FileStore/shared_uploads/michaelkennethkingston@gmail.com/output'

In [0]:
# not needed in databricks but needed elsewhere
# sc = SparkContext()

In [0]:
# read the textfile & check it
lines = sc.textFile(abc_news)
lines.take(5)

Out[39]: ['20030219,council chief executive fails to secure position',
 '20030219,council welcomes ambulance levy decision',
 '20030219,council welcomes insurance breakthrough',
 '20030219,fed opp to re introduce national insurance',
 '20040501,cowboys survive eels comeback']

In [0]:
# iterate through each line. split on commas
# this is basically the function wilson write for week 4 with a slight tweak

def dates_func(lines):
    ts, text = lines.split(',')
    year = ts[:4]
    words = text.split()
    for word in words:
        return year, (word, 1)

In [0]:
# this is a modified version of the function from week 4 written by wilson.
def second_func(year, word_count_pairs):
    year_count = {}
    for word, count in word_count_pairs:
        if word not in word_count:
            word_count[word] = count
        else:
            word_count[word] += count

    max_word_count_pair = sorted(year_count.items(), key=lambda x: (x[1], x[0]))[-1]
    word = max_word_count_pair[0]
    count = max_word_count_pair[1]
    return year, f"{word}:{count}"

In [0]:
# This gives me a reasonable return. I just need to aggregate on word, then group by year. second_func should work but I can't call it correctly...
lines = sc.textFile(abc_news).map(dates_func)
lines.take(5)

Out[44]: [('2003', ('council', 1)),
 ('2003', ('council', 1)),
 ('2003', ('council', 1)),
 ('2003', ('fed', 1)),
 ('2004', ('cowboys', 1))]

In [0]:
# how do i call second_func? This should work but it doesn't.
pairs = lines.reduceByKey(second_func)
pairs.take(5)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-3021736009984663> in <module>
      1 # how do i call it?
      2 pairs = lines.reduceByKey(second_func)
----> 3 pairs.take(5)

/databricks/spark/python/pyspark/rdd.py in take(self, num)
   1598 
   1599             p = range(partsScanned, min(partsScanned + numPartsToTry, totalParts))
-> 1600             res = self.context.runJob(self, takeUpToNumLeft, p)
   1601 
   1602             items += res

/databricks/spark/python/pyspark/context.py in runJob(self, rdd, partitionFunc, partitions, allowLocal)
   1333             finally:
   1334                 os.remove(filename)
-> 1335         sock_info = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
   1336         return list(_load_from_socket(sock_info, mappedRDD._jrdd_deserializer))
   1337 

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_g

In [0]:
# this is based on an example i saw but it doesn't do what i want
# I need to group & sum the values before doing reduceByKey() to get the result I'm looking for
# the below doesn't work

from operator import add

pairs = lines.reduceByKey(add)
print(pairs.collect())

[('2003', ('council', 1, 'council', 1, 'council', 1, 'fed', 1)), ('2004', ('cowboys', 1, 'cowboys', 1, 'castro', 1)), ('2020', ('coronanomics', 1, 'coronavirus', 1, 'coronavirus', 1, 'coronavirus', 1, 'china', 1))]


In [0]:
# count the total number of words


In [0]:
# groupBy(year)


In [0]:
# sort into top 3 per year with the format <year    word[0] word[1] word[2]>


In [0]:
# save the output
# result.saveAsTextFile('output.txt')

In [0]:
# close sc()


In [0]:
# show file


In [0]:
# run the program
# spark-submit rdd.py